In [21]:
from astropy.io import fits
import numpy as np

In [22]:
def stackFits(fileName,startNum,stopNum,nameOnly=False):
    fileList = []
    nFill = max([len(str(startNum)),len(str(stopNum))])
    for n in range(startNum,stopNum+1):
        nName = fileName % str(n).zfill(nFill)
        try:
            file = fits.getdata(nName)
            if nameOnly:
                fileList.append(nName)
            else:
                fileList.append(file)
        except FileNotFoundError:
            pass
    if len(fileList) != stopNum-startNum+1:
        print(str(len(fileList))+'/'+str(stopNum-startNum+1),'files stacked')
    return fileList

In [23]:
def medianFits(fileName,startNum,stopNum):
    stack = stackFits(fileName,startNum,stopNum)
    medianStack = np.median(stack,axis=0)
    return medianStack

In [36]:
def calibrate(fileList,medianBias,medianDark,medianFlat):
    alreadyCalibrated = []
    newCalibrations = []

    for file in fileList:
        newFileName = file[-37:]
        data,header = fits.getdata(file,header=True)
        data = (data - medianBias - medianDark) / medianFlat
        hdu = fits.PrimaryHDU(data,header=header)
        try:
            hdu.writeto('Data/Calibrated/r' + newFileName)
            newCalibrations.append('r'+newFileName)
        except OSError:
            alreadyCalibrated.append('r'+fileName)
        
    print("Successfully calibrated:",len(newCalibrations),"of",len(fileList))
    print("Already calibrated:",len(alreadyCalibrated),"of",len(fileList))

### October 12

In [25]:
biasName = 'Data/Uncalibrated/20181012p3/lux-180.000secs.Bias.Open.000000%s.fit'
medianBias = medianFits(biasName,46,55)

In [26]:
darkName = 'Data/Uncalibrated/20181012p3/lux-180.000secs.Dark.Open.000000%s.fit'
medianDark = medianFits(darkName,41,45)
medianDark -= medianBias

In [27]:
medianFlat = fits.getdata('Data/Flats/p3/openflat-1005-1x1.fit')

In [28]:
fileName = 'Data/Uncalibrated/20181012p3/lux-180.000secs.Light.Open.000000%s.fit'
imgList = stackFits(fileName,15,38,nameOnly=True)

In [34]:
calibrate(imgList,medianBias,medianDark,medianFlat)

Successfully calibrated: 24 of 24
Already calibrated: 0 of 24


### October 17

In [38]:
biasName = 'Data/Uncalibrated/20181017p3/a-180.000secs.Bias.Open.00000%s.fit'
medianBias = medianFits(biasName,206,215)

In [39]:
darkName = 'Data/Uncalibrated/20181017p3/a-240.000secs.Dark.Open.000000%s.fit'
medianDark = medianFits(darkName,36,38)
medianDark -= medianBias

In [40]:
medianFlat = fits.getdata('Data/Flats/p3/openflat-1005-1x1.fit')

In [41]:
fileName = 'Data/Uncalibrated/20181017p3/a-240.000secs.Light.Open.000000%s.fit'
imgList = stackFits(fileName,12,35,nameOnly=True)

22/24 files stacked


In [42]:
calibrate(imgList,medianBias,medianDark,medianFlat)

Successfully calibrated: 22 of 22
Already calibrated: 0 of 22
